In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
print module_path
if module_path not in sys.path:
    sys.path.append(module_path)

/home/ryan/Projects


In [3]:
import pandas as pd
import numpy as np
import pymongo
import random
from pymongo import MongoClient
from datetime import datetime, timedelta
from sshtunnel import SSHTunnelForwarder
import sshtunnel
pd.set_option('display.max_colwidth', -1)

In [4]:
def mongo_connect(mongo_port=27020):
    # Try to connect to MongoDB,  exit if not successful.
    try:
        conn = MongoClient('localhost', mongo_port)
        # print "Connected successfully to MongoDB, instance created!"

    except pymongo.errors.ConnectionFailure, e:
        print "Could not connect to MongoDB: %s" % e

    name = 'SA_Voting_Data'

    # create database on instance
    if name in conn.database_names():
        # print "Found SA_Voiting_Data Database"
        # print "Connecting to db..."
        db = conn[name]  # Create the database

        # create one collection, called conversations
        conversations = db.conversations_collection
        raw = db.raw_collection
        # print ("Connected!")
    else:
        # print "Creating SA_Voting_Data Database"
        db = conn[name]  # create the database
        conversations = db.conversations_collection
        raw = db.raw_collection

    # collection is a collection in the mongodb instance created above
    # db is the database on the mongodb instance
    # conn is the mongodb instance
    return conn, db, conversations, raw

def get_conversation(conversations):
    header = ['content','from_addr','to_addr','transport_type','transport_name','timestamp','session_event']
    df_conversations = pd.DataFrame()
    
    for c in conversations:
        data = [c['content'],c['from_addr'],c['to_addr'],c['transport_type'],c[u'transport_name'], c['timestamp'],c['session_event']]
        df_conversations = df_conversations.append([data])
    
    df_conversations.columns = header
    df_conversations = df_conversations.set_index('timestamp')
    return df_conversations


def alpha(conversation_df):
    """average number of responses given per open session and total number of sessions"""
    total_number_of_responses = len(conversation_df[conversation_df['session_event']=='resume'])
    number_of_sessions = len(conversation_df[conversation_df['session_event']=='close'])
    if number_of_sessions == 0:
        return 0, 0 
    else:
        return float(total_number_of_responses)/float(number_of_sessions), float(number_of_sessions)


def get_total_time_in_system(df):
    if len(df[df['session_event'] == 'new']) != 0:
        # start is the first instance we observe a new_connection
        start = datetime.strptime(df[df['session_event'] == 'new'].index[0], TIME_STAMP_FORMAT)
        # finish is the very last report of the system
        finish = datetime.strptime(df.index[-1],TIME_STAMP_FORMAT)
        delta = finish - start
        return delta.seconds
    else:
        return 0


def beta(conversation_df):
    """Returns total time interacting and total time in system """
    total_time_in_system = get_total_time_in_system(conversation_df)
    total_time_interacting = 0.0
    start_time = 0.0
    finish_time = 0.0
    in_session = False
    for rows in conversation_df.iterrows():

        if rows[1]['session_event'] == 'new':
            start_time = datetime.strptime(rows[0],TIME_STAMP_FORMAT)
            in_session = True

        if rows[1]['session_event'] == 'close' and in_session is True:
            finish_time = datetime.strptime(rows[0],TIME_STAMP_FORMAT)
            delta = finish_time - start_time
            time_interacting_in_session = delta.seconds
            total_time_interacting += delta.seconds
            in_session = False
            
    return total_time_interacting, total_time_in_system

def surfing(conversation_df, address):
    q = conversation_df[conversation_df['to_addr'] != address]['to_addr']
    if len(set(q[q  != u'None' ])) > 1:
        return True
    else:
        return False

In [5]:
# constants here
CONNECT = mongo_connect
TIME_STAMP_FORMAT = "%Y-%m-%d %H:%M:%S.%f"

In [6]:
conn, db, conversation, raw = CONNECT()
try:
    # if you want to make queries to the db, do it here
    pois = list(conversation.distinct('poi'))
    types_of_events = raw.distinct('session_event')
finally:
    conn.close()

In [ ]:
len(pois) # number out of 260k

In [10]:
conn, db, conversation, raw = CONNECT()

try:
    # if you want to make queries to the db, do it here
    conversations = list(conversation.find({'poi': { '$in': pois[0:1000] }}))
finally:
    conn.close()

conversation_data_frame = get_conversation(conversations)

In [11]:
len(conversations)

33309

In [ ]:
# FOR TESTING
# for sample in pois[0:1000]:
#     sample_conversation = []
#     for c in conversations:
#         if c['poi'] == sample:
#             sample_conversation.append(c)
    
#     conversation_data_frame = get_conversation(sample_conversation)
    
#     print len(conversation_data_frame)
#     print alpha(conversation_data_frame)
#     print beta(conversation_data_frame)
#     print surfing(conversation_data_frame, sample)

In [12]:
import csv
with open('out.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['id', 'length', 'a1', 'a2', 'b1', 'b2', 'surfing'])
    
    for sample in pois[0:1000]:
        sample_conversation = []
        for c in conversations:
            if c['poi'] == sample:
                sample_conversation.append(c)
        my_id = sample
        conversation_data_frame = get_conversation(sample_conversation)
        a1, a2 = alpha(conversation_data_frame)
        b1, b2 = beta(conversation_data_frame)
        surfing_var = surfing(conversation_data_frame, sample)
        length = len(conversation_data_frame)
        spamwriter.writerow([my_id, length, a1, a2, b1, b2, surfing_var])